# This is my FPL analysis workbook

In [14]:
#!/usr/bin/env python
# coding: utf-8
import os
import numpy as np
import pandas as pd
import requests
from datetime import datetime
import json
from tqdm import tqdm

In [15]:
# Variables
season = "2020-21"

## Set display options

In [16]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [17]:
#### Connect to FPL API
def get(url):
    response = requests.get(url)
    return json.loads(response.content)

#### Getting player summary data
def get_player_summary(player_id):
    url = 'https://fantasy.premierleague.com/api/element-summary/' + str(player_id) + '/' 
    response = get(url)
    fixtures = response['fixtures']
    history = response['history']
    history_past = response['history_past']
    return [fixtures, history, history_past]

## Dimension data from FPL

In [18]:
link = 'https://fantasy.premierleague.com/api/bootstrap-static/'
data = get(link)

In [19]:
print(data.keys())

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])


In [20]:
# All player stats (total)
all_players = pd.DataFrame(data['elements'])

# All tema stats (total)
all_teams = pd.DataFrame(data['teams'])

# All gameweek stats
all_events = pd.DataFrame(data['events'])

# Player types
player_types = pd.DataFrame(data['element_types'])

# Game setting
game_phases = pd.DataFrame(data['phases'])



## Fact data from FPL

In [21]:
player_season_history_list= []
player_gameweek_history_list = []
player_future_fixture_list = []

for i in tqdm(range(len(all_players))):
    try:
        fixtures, history, history_past = get_player_summary(i)
        for j in range(len(fixtures)):
            fixtures[j]['player_id'] = i
        player_gameweek_history_list.append(history)
        player_season_history_list.append(history_past)
        player_future_fixture_list.append(fixtures)
    except KeyError:
        continue


100%|██████████| 597/597 [00:36<00:00, 16.35it/s]


In [22]:
player_season_history = [item for sublist in player_season_history_list for item in sublist]
player_season_history = pd.DataFrame(player_season_history)

player_gameweek_history = [item for sublist in player_gameweek_history_list for item in sublist]
player_gameweek_history = pd.DataFrame(player_gameweek_history)

player_future_fixture = [item for sublist in player_future_fixture_list for item in sublist]
player_future_fixture = pd.DataFrame(player_future_fixture)


## Save Raw data files

In [23]:
all_players.to_csv (r'./data/' + season + '/Raw_data/all_players.csv', index = False, header=True)
all_teams.to_csv (r'./data/' + season + '/Raw_data/all_teams.csv', index = False, header = True)
all_events.to_csv (r'./data/' + season + '/Raw_data/all_events.csv', index = False, header = True)
player_types.to_csv(r'./data/' + season + '/Raw_data/player_types.csv', index = False, header = True)
game_phases.to_csv(r'./data/' + season + '/Raw_data/game_phases.csv', index = False, header = True)

player_season_history.to_csv (r'./data/' + season + '/Raw_data/player_season_history.csv', index = False, header=True)
player_gameweek_history.to_csv (r'./data/' + season + '/Raw_data/player_gameweek_history.csv', index = False, header=True)
player_future_fixture.to_csv (r'./data/' + season + '/Raw_data/player_future_fixture.csv', index = False, header=True)